<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Yield Curve Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>().ResetType<YieldCurve>());

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioYieldCurve = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "YCUP1.0pct", ImportFormats.Cashflow);
var argsScenarioTransactionalData = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "SRUP1.0pct", ImportFormats.Cashflow);

var ws = Workspace.CreateNew();
ws.InitializeFrom(DataSource);
var initStorage = new ImportStorage(argsBestEstimate, DataSource, ws);
await initStorage.InitializeAsync();

var actualETs = initStorage.EstimateTypesByImportFormat[ImportFormats.Actual];
var cashflowETs = initStorage.EstimateTypesByImportFormat[ImportFormats.Cashflow];
var actualOrCashflowETs = actualETs.Intersect(cashflowETs);
var onlyActualETs = actualETs.Except(cashflowETs);
var onlyCashflowETs = cashflowETs.Except(actualETs);

In [0]:
var log = await Import.FromFile("../Files/Parameters/YieldCurve_2019_12.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_1.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_3.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_12.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_3.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_6.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_12.csv").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync());
log

# Best Estimate Initialization

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
/*var file = "scenarioYieldCurve.csv";
var extension = System.IO.Path.GetExtension(file);

var stream = await Project.FileStorage.ReadAsync(file);

var dataSet = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet;
    Activity.Start();
    var args = GetArgsFromMain(dataSet) with {ImportFormat = ImportFormats.YieldCurve};
    args.ValidateArgsForPeriod();
    if(ApplicationMessage.HasErrors())  return Activity.Finish();

    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<YieldCurve>());

    var yieldCurves = await DataSource.Query<YieldCurve>().ToArrayAsync();
    var hasNameColumn = dataSet.Tables[ImportFormats.YieldCurve].Columns.Any(x => x.ColumnName == nameof(YieldCurve.Name));
    var importLog = await Import.FromDataSet(dataSet).WithType<YieldCurve>((dataset, datarow) =>  { 
                var yieldCurve = new YieldCurve {
                Currency = datarow.Field<string>(nameof(YieldCurve.Currency)),
                Year = args.Year,
                Month = args.Month, 
                Scenario = args.Scenario,
                Values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(YieldCurve.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray(),
                Name = hasNameColumn
                            ? datarow.Field<string>(nameof(YieldCurve.Name))
                            : default(string)
            };
            return yieldCurves.Contains(yieldCurve, YieldCurveComparer.Instance())
                    ? null
                    : yieldCurve;
            }
        ).WithTarget(workspace).ExecuteAsync(); 
    
    if(importLog.Errors.Any()) return Activity.Finish().Merge(importLog); 
    yieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    if (!yieldCurves.Any()){
        ApplicationMessage.Log(Warning.VariablesAlreadyImported);   
        return Activity.Finish().Merge(importLog);
    }
    
    var calculationLog = await CalculationYieldCurvesAsync(args, workspace);
    if(calculationLog.Errors.Any()) return Activity.Finish().Merge(importLog).Merge(calculationLog); 
    //await DataSource.UpdateAsync<YieldCurve>(yieldCurves);
    //await DataSource.CommitAsync();
    return Activity.Finish().Merge(importLog).Merge(calculationLog);*/

# Scenario Yield Curve Import

In [0]:
var yieldCurveScenario = @"
@@Main
Year,Month,Scenario
2020,12,YCUP1.0pct
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
EUR,0.108685495,0.108685495,0.118498471,0.119966839
CHF,0.102,0.102,0.102,0.102
USD,0.102,0.102,0.102,0.102
";

In [0]:
await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurve = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = (ivsScenarioYieldCurve.Where(x => Math.Abs(x.Value) > Precision).ToArray()).Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualETs.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowETs.Contains(x.EstimateType)).ToArray();
var actualOrCashflowVars = diffs.Where(x => actualOrCashflowETs.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().NotBe(0);
diffs.Except(actualVars.Union(cashflowVars).Union(actualOrCashflowVars)).ToArray().Length.Should().Be(0);

# Further Import of the same Yield Curve should not retrigger IFRS17 Calculations

In [0]:
var log = await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync();
var ivsScenarioYieldCurveReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioYieldCurve.Except(ivsScenarioYieldCurve, IfrsVariableComparer.Instance());
diffs.ToArray().Length.Should().Be(0);

# Scenario Transactional Data Import

In [0]:
var cashflowScenarioTransactionalData = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,SRUP1.0pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,CU,CL,C,,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-11,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-5.5,-53.5,-52.5,-31.3
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowScenarioTransactionalData).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalData = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var bestEstimateYieldCurve = @"
@@Main
Year,Month
2020,12
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
EUR,0.108685495,0.108685495,0.118498471,0.119966839
CHF,0.102,0.012,0.102,0.102
USD,0.102,0.102,0.102,0.102
GBP,0.102,0.102,0.102,0.102
";

In [0]:
await Import.FromString(bestEstimateYieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateYieldCurve = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = (ivsBestEstimateYieldCurve).Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
diffs.ToArray().Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalDataRecalculation = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioTransactionalDataRecalculation.Except(ivsScenarioTransactionalData, IfrsVariableComparer.Instance());
diffs.Where(x => Math.Abs(x.Value) > Precision).ToArray().Length.Should().NotBe(0);

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioTransactionalDataRecalculation = await DataSource.Query<IfrsVariable>().ToArrayAsync();
var diffs = ivsScenarioTransactionalDataRecalculation.Except(ivsScenarioTransactionalData, IfrsVariableComparer.Instance());
diffs.Where(x => Math.Abs(x.Value) > Precision).ToArray().Length.Should().NotBe(0);